In [27]:
import pandas as pd
import numpy as np
import re

from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [93]:
def join_words(text):
    interm =  ','.join(text)
    return interm.replace(",", " ")

def flatten(xss):
    return [x for xs in xss for x in xs]

def extract_topics(lda_model):
    a = [topic for idx, topic in lda_model.print_topics(-1)]
    r = flatten([i.split('+') for i in a])

    return [re.findall('"([^"]*)"', i)[0] for i in r]

In [9]:
reviews = pd.read_excel('../alfa_bank_reviews_all.xlsx')

In [13]:
reviews['text'].sample(7).values

array(['Кошмар, нужно было оплатить в обход ареста кредит, на что мне начали зачем то рассказывать что нужно вам карту нашего банка, рассксзывала дама в черном пиджаке без бейджика, кто мог решить вопрос был занят, прождав 20 минут результата я не получил ушел, в отделении работают 3 человека',
       'Решила покопить, пока работа у меня стабильная есть)))открыла в этом банке совсем недавно Альфа - счёт. Ставку одобрили под 7 процентов, и срок у него бессрочный. Хочешь докладывай, хочешь снимай, от этого сильно процент не пострадает.',
       'мало людей, соответственно быстрое обслуживание.',
       '3 менеджера на огромный поток клиентов, ожидание по 2 часа, чтобы выполнить процедуру получения карты за 2-3 минуты, это просто издевательство. Спасибо за убитое время!',
       'Хорошее комфортное место, был приятный менеджер (Александр если не ошибаюсь). Все подробно и грамотно рассказал. Но единственное, девушка на входе не совсем понятно объясняет что и как нужно делать. Поэтому 4 зве

In [ ]:
# TODO: 'кредит_наличными' - > кредит 

In [ ]:
# tags 
# 'банкоматы' 'без_тематики' 'график_работы' 'ипотека' 'карта'
#  'кассовое_обслуживание' 'комфорт' 'контактный_центр' 'кредит_наличными'
#  'мобайл/онлайн' 'мошенничество' 'навязывание_продуктов' 'очередь'
#  'парковка' 'скорость_работы' 'сотрудники' 'счет'

In [121]:
def create_tags(review):
    if type(review) == str:
        if 'очеред' in review or 'долго' in review  or 'медлен' in review or 'ожидан' in review or 'минут' in review \
         or 'народу' in review:
            return 'очередь'
        elif 'сотрудн' in review or 'грамотн' in review or  'вежлив' in review or 'персонал' in review \
        or 'коллектив' in review or 'менеджер' in review or 'обслужив' in review \
        or 'работник' in review or 'специалист' in review or 'девочк' in review or 'оперативно' in review or 'помогли' in review:
            return 'сотрудники'
        elif 'банкомат' in review:
            return 'банкоматы'
        elif 'график' in review:
            return 'график_работы'
        elif 'кредит' in review:
            return 'кредит'
        elif 'ипотек' in review:
            return 'ипотека'
        elif 'карт' in review:
            return 'карта'
        elif 'касс' in review or 'валют' in review :
            return 'кассовое_обслуживание'
        elif 'комф' in review or 'удобн' in review :
            return 'комфорт'
        elif 'мошенн' in review or 'обман' in review :
            return 'мошенничество'
        elif 'быстр' in review or 'скорость' in review :
            return 'скорость_работы'
        elif 'счет' in review or 'счёт' in review:
            return 'счет'
        elif 'навяз' in review or 'впарил' in review:
            return 'навязывание_продуктов'
        
        elif 'парков' in review:
            return 'парковка'
        else:
            return None
    else:
            return None

# TODO: синонимы и опечатки

In [122]:
reviews['text'] = reviews['text'].str.lower()
reviews['text']  = reviews['text'] .apply(simple_preprocess) 
reviews['text']  = reviews['text'] .apply(join_words) 

reviews['re_tag'] = reviews['text'].apply(create_tags)

In [123]:
len(reviews[reviews['re_tag'].notnull()]) / len(reviews)

0.8268594340022911

In [133]:
pd.set_option('display.max_colwidth', None)
reviews[reviews['re_tag'].notnull()][['text', 're_tag']].sample(7)

,text,re_tag
31495,только положительные эмоции после посещения этого отделения заезжала забрать свою карту сотрудник вежливый все быстро оформил помог так же подсказал про бонусы от банка спасибо,карта
70341,оформила кредитную карту он лайн срок ожидание был дня пришла смс готовности что могу получить карту доставка была это отделение сожалению это отделение банка просто своим отношением отбило все желание стать клиентом первый день сказали ожидание минимум минут ждать не стала пришла на следующий день ожидание минут дождалась своей очереди надежде забрать свою карту несколько раз сказала менеджеру который ко мне подошёл что заказывала именную карту переспросила почему не могу получить свою если пришла смс готовности молчание невнятный ответ из серии не знаю как сотрудник банка может не знать ответа на такой элементарный вопрос итог карту мне не отдали причину не объяснили зато менеджер очень быстренько решил оформить карту заново не сказав мне об этом часа потраченного времени пустую не советую это отделение банка звезда за приветливую девушку на входе за оператор по телефону,очередь
83625,отделение рядом метро очередь идёт достаточно быстро вежливые компетентные сотрудники,очередь
905,хороший банк обслуживание мне понравилось,сотрудники
9334,удобно быстро вежливо,комфорт
1147,потеряла банковскую карту неподалеку от этого отделения сотрудники оперативно заблокировали карту дали советы оформили новую проявив заботу внимание,карта
1555,навязывают не нужные опции если открываете счёт сразу узнавайте всё снять деньги карты причём банкомате процентом считаю что не удобный банк,банкоматы


In [125]:
reviews[reviews['re_tag'].isnull()][['text']].sample(7)

,text
56553,хорошее отделение давно там не было но только положительные моменты сохранились работе этого отделения
29609,на высоте офис небольшой
58874,хорошее расположение
74103,лучший клиентский опыт каждый раз получаю данном отделении банка рад приходить сюда снова снова рекомендую альфа банк как лучший банк для физ лиц
55439,хороший банк
8875,надежный банк где не возникало проблем вкладами процентами момента его образования
84816,приезжал несколько раз привозил платёжки от своего ооо так ни разу их не провели не было человека который занимался бы юрлицами больше алексин ни ногой


In [126]:
# TODO: сравнить длину отзывов яндекса и геосервисов

#### тематическое моделирование

In [127]:
documents = reviews[reviews['re_tag'].isnull()]['text'].to_list()
documents[1]

'оставляю положительный отзыв банке после нескольких обращений вопросами на которые всегда адекватно полноценно отвечали'

In [128]:
# TODO: список стоп слов 
with open('../stop-words-ru.txt') as f:
    russian_stopwords = [x.strip('\n') for x in f]

In [129]:
# отделение офис уютно в комфорт?
# классы пенсия приложение вклад 



In [130]:
russian_stopwords.extend([ 'банк', 'банка', 'альфа', 'втб', 'сбер', 'зеленый', 'отлично', 'супер', 'хорошее', 'молодец', 'работает', 
                          'пользуюсь', 'рекомендую', '',
                         'вопрос', 'понравилось', 'отношение', 'высшем', 'работают', 'хочу', 'итоге', 'дозвониться', 'пришлось',
                         'ужасный', 'нравится' , 'приятно', 'отличный'])

In [131]:
%%time
# stop_words = set(STOPWORDS)
texts = [[word for word in simple_preprocess(doc) if word not in russian_stopwords] for doc in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Обучение модели LDA
num_topics = 5
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
# for idx, topic in lda_model.print_topics(-1):
#     print(f"Тема {idx + 1}: {topic}")

topics = extract_topics(lda_model)
topics

CPU times: user 19.8 s, sys: 38.1 ms, total: 19.8 s
Wall time: 19.9 s


['отделение',
 'лучший',
 'условия',
 'сервис',
 'норм',
 'клиентов',
 'отличное',
 'плохо',
 'выгодные',
 'банком',
 'офис',
 'современный',
 'большое',
 'нормальный',
 'клиент',
 'отделении',
 'обычный',
 'девушки',
 'уютно',
 'устраивает',
 'советую',
 'клиентам',
 'вклад',
 'клиентом',
 'банке',
 'уровне',
 'банков',
 'условия',
 'лучших',
 'своим',
 'молодцы',
 'работы',
 'атмосфера',
 'формат',
 'клиентами',
 'ребята',
 'клиенту',
 'понятно',
 'банки',
 'хорошая',
 'невозможно',
 'банком',
 'нормально',
 'ок',
 'вопросы',
 'банке',
 'высоте',
 'проблем',
 'работы',
 'приложение']

In [132]:
# with open('../stop-words-ru.txt', 'w') as f:
#     for line in russian_stopwords:
#         f.write(f"{line}\n")